## Импорт библиотек

In [1]:
# импорт библиотек
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

import psycopg2
from yaml import load, FullLoader
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine


## Парсинг

In [2]:
# ссылка
URL = 'https://www.cbr.ru/currency_base/daily/'  

# User-Agent
headers = {
    "Accept": "*/*",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
}

# сохраняем объект Response в переменную req
req = requests.get(URL, headers=headers)  

# текстовое содержание запроса
req_text = req.text  

# парсинг
soup = BeautifulSoup(req_text, 'lxml')  

table = soup.find('table')  # таблица

heading_table = []  # список, в котором будут храниться названия столбцов

# цикл для поиска нащвания столбцов и добавление в список
for row in table.find_all('th'):
    heading_table.append(row.text)
print(heading_table)



['Цифр. код', 'Букв. код', 'Единиц', 'Валюта', 'Курс']


In [3]:
content=[] # список, в котором будут храниться данные из таблицы

# цикл для добавление информации построкам
for row in table.find_all('tr'): 
    if not row.find_all('th'): # пропустить первую строку таблицы с заголовками
            content.append([element.text for element in row.find_all('td')])

In [4]:
# создание DataFrame
rate = pd.DataFrame(content, columns=heading_table) 
rate['date'] = datetime.now().date() # создание столбца с текущей датой

# переименование столбцов
rate.columns = ['id', 'literal_id', 'unit', 'curr_name', 'curr_rate', 'date']

# замена запятой на точку
rate['curr_rate'] = rate['curr_rate'].str.replace(',', '.') 

display(rate.head()) 

,id,literal_id,unit,curr_name,curr_rate,date
0,036,AUD,1,Австралийский доллар,40.9750,2022-11-27
1,944,AZN,1,Азербайджанский манат,35.5763,2022-11-27
2,051,AMD,100,Армянских драмов,15.3094,2022-11-27
3,933,BYN,1,Белорусский рубль,25.0320,2022-11-27
4,975,BGN,1,Болгарский лев,32.1992,2022-11-27


## Создание базы и таблицы в PostgreSQL

**Создание базы**

In [5]:
# # импорт параметров для подключения к базе
# with open('config_postgresql.yaml') as f:
#     config = load(f, Loader=FullLoader)

# # подключение к базе
# connection = psycopg2.connect(
#             host=config['host'],
#             database='',
#             user=config['user'],
#             password=config['password'],
#             port = config['port']
#         )
# connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# # курсор для выполнения операций с базой данных
# cursor = connection.cursor()

# # создание базы данных
# cursor.execute('CREATE DATABASE cbr_rate;')
# connection.commit()

# # закрытие сессии
# connection.close()

**Создание таблицы**

In [6]:
# # подключение к базе
# connection = psycopg2.connect(
#             host=config['host'],
#             database='cbr_rate',
#             user=config['user'],
#             password=config['password'],
#             port = config['port']
#         )
# connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# def sql_query(query, connection):
#     """Отправка SQL-запроса query"""
#     cursor = connection.cursor()
#     cursor.execute(query)
#     connection.commit()

# # создание таблицы
# create_table = """
# CREATE TABLE cbr_rate_table (
#     id           integer,
#     literal_id   text,
#     unit         integer,
#     curr_name    text,
#     curr_rate    float4,
#     date         date    
# )
# """

# sql_query(create_table, connection=connection)

# # закрытие сессии
# connection.close()

## Заливка данных в таблицу

In [7]:
# импорт параметров для подключения к базе
with open('config_postgresql.yaml') as f:
    config = load(f, Loader=FullLoader)
    
# подключение к базе данных
engine = create_engine(f"postgresql+psycopg2://{config['query_bd']}")

# заливка данных
rate.to_sql('cbr_rate_table', engine, if_exists='append', index=False)

# закрытие сессии
connection.close()